<a href="https://colab.research.google.com/github/russamy2010/databasics/blob/main/Data%20Cleansing%2C%20but%20Make%20it%20(in)%20Colab%20Part%201%20Structured_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleansing, but Make it (in) Colab: Part 1

Hello! This notebook will give you an opportunity to get a grasp of data cleansing. Data cleansing is an integral step in any data analytics projects, since without clean data, you won't have a good data analytics project. You can use this notebook as a guide as well as a reference when trying to determine a starting course of action when cleansing data.

To prevent this notebook from being too bulky, not every technique will (and can be) covered one notebook. The techniques covered in this notebook will be a combination of the most common (i.e., one-hot encoding) and ones that are very useful to have in a data analyst's tool kit. I do recommend that you research techniques not covered here to determine if they are a good fit your use case.

# Introduction

Data cleansing can be a complicated process. Partly because of execution, but partly because of the need to determine WHICH data cleansing technique to use. One of the primary determinats of this understanding the data and possible issues with that data and how the issues can effect the end use.

Outside of that, you can arguably classify data cleanings techniques in three ways:
  1. Data Classification
  1. End Goal
  1. Data Structure

This notebook is organized as follows:

## Outline
  1. Data Cleansing by Data Classification
  1. Data Cleansing by End Goal
  1. Data Cleansing by Data Structure

Each section will contain information about the data cleansing methods and will be followed by examples of 1-2 of the data cleansing methods mentioned in that section.

Now, let's get started!

## Setup
First, let's import useful libraries and convenience functions for our Pandas Dataframe.


In [ ]:
# @title Import Libraries and Functions
!pip install tabulate
from __future__ import print_function

import numpy as np
import pandas as pd
import collections
from matplotlib import pyplot as plt
import sklearn
import seaborn as sns
import altair

from google.colab import files
import zipfile

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler


# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.3f}'.format
def mask(df, key, function):
  """Returns a filtered dataframe, by applying function to key"""
  return df[function(df[key])]

def flatten_cols(df):
  df.columns = [' '.join(col).strip() for col in df.columns.values]
  return df

pd.DataFrame.mask = mask
pd.DataFrame.flatten_cols = flatten_cols


# Install the Kaggle library (only needed for the first time you run this)
!pip install -q kaggle



Next, let's import some data. We are going to use the NBA dataset from Kaggle located at this link: [NBA Dataset](https://www.kaggle.com/datasets/wyattowalsh/basketball/data).

In [ ]:
# @title Load the NBA data in dataframes (run this cell).

"""Downloads the NBA Dataset and creates the Pandas Data Frames.
  Args:
    df: a dataframe.
    files_upload: there will be an option for you to upload your Kaggle API key.
    To get your API performing the following steps:
       1.) Log into your Kaggle Account
       2.) Go to your Settings-->Account and scroll to the 'API section
       3.) Click "Create New Token" the token file will download
    in the display box, click the grey 'Choose file' button to upload your API key
  """

# Upload your kaggle.json file
uploaded = files.upload()

# Make the kaggle directory and move the kaggle.json file there
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Change the permissions of the file
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d wyattowalsh/basketball -p ./

# Unzip the dataset
with zipfile.ZipFile('basketball.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

# Get a list of all CSV files in the extracted directory
import os
csv = [f for f in os.listdir('/content/csv') if f.endswith('.csv')]

# Create a dictionary to store the DataFrames
dataframes = {}

# Read each CSV file into a separate DataFrame
for csv_file in csv:
    df_name = os.path.splitext(csv_file)[0]  # Use the filename (without extension) as the DataFrame key
    file_path = os.path.join('/content/csv', csv_file)  # Construct the full file path
    dataframes[df_name] = pd.read_csv(file_path)
    print(f"Loaded {csv_file} into {df_name}")





In [ ]:
#@title Dataframe Check
"""This code is to check the dataframes.
 Replace 'df_name' with the actual DataFrame name i.e., draft_history to
  view the 'draf_history.csv' Dataframe.
  You don't have to run this code, but it is available in case you want to check
    the outputs of the dataframes.
  """
# Access the individual DataFrames
print(dataframes['game'].head())

# I. Techniques by Data Classification
Different types may require (or be more predisposed to) being cleansed using different methods. So, understanding data classification is key. We can classify data into three primary data tyes.

1. Numerical: data that is presented in numbers and can be used for mathematical and arithmetical calculations.
1. Categorical (this has two subsets--ordinal and nominal): represent types of data which may be divided into groups. It if is ordinal, that means order matters (i.e., first, second, third) and if it nominal abels variables into categories without using numbers or providing a hierarchy
1. Other data: Examples of this data type include text, photo, videos, etc. Text is data  also known as character strings, is any information or message conveyed in written or printed form; photo data is a combination of image data and its metadata, which is information about the image and its production; and video data is data is visual information captured by cameras (this type of data is often called unstructured data).






###Level setting

Before moving on to techniques, it is important to level set. Since there is so many  techniques, it is necessary to limit the scope of this notebook (or else it would become much too long!). As such, this notebook will focuse on numerical and categorical data and the techniques used for those data types. A part 2 of this notebook will focus on unstructured data and techniques relevant to those types.

###The Meat and Potatoes: Techniques

Considering the above, we can take different data techniques and place them into groups that focus on key issues found within each data classification:


Numerical Data
1. Handling Missing Values
*  Imputation
*  Deletion
*  MICE (Multiple Imputation by Chained Equations)

2. Outlier detection and treatment
*   Statistical methods
*   Visualization
*   Winsorization
*   Transformation

3. Data transformation and standardization
*   Scaling
*   Normalization
*   Log Transformation
*   Binning

Categorical Data:
1. Handling Missing Values
*   Mode imputation
*   Creating a new "missing" category
*   Deletion (if infrequent)

2. Inconsistent Values
*   Standardization
*   Mapping to a controlled vocabulary


3. Encoding
*   One-hot encoding
*   Label encoding
*   Ordinal encoding (if order matters)

(Note: Seems like handling missing values is a common issue across these data types.)


### Numerical Data: Handling Missing Values Examples
Let's first start with Numerical Data and examples of implementing techniques for handling missing values.

In [ ]:
# @title Numerical Data Example for Handling Missing Values: Imputation using Mean

"""
Strategy Explanation: Why Mean and not Median, Most Frequent (Mode)?
   This example uses the Median to fix the issue of missing values for the arena capacity column.
   You do not have to use mean: other strategies include using the Median or Most_Frequent (i.e., Mode)
   Median is used here because then you would assume the variable has a skewed distribution (which is possible, given arena
    size can be influenced by quite a bit of factors, including popularity of the sport)
   If you want to use Mean, then you would assume the variable has a Gaussian distribution.
   Mode is mostly used for categorial data, as mentioned in the above section
"""

from sklearn.impute import SimpleImputer

# Convert the dictionary to a DataFrame
df_update_teamdetails = pd.DataFrame(dataframes['team_details'])

# Initialize the impute
imputer = SimpleImputer(strategy='median')

# Fit and transform the DataFrame to fill missing values
df_update_teamdetails["arenacapacity"] = imputer.fit_transform(df_update_teamdetails[["arenacapacity"]])

# Display the updated DataFrame before converting back to a dictionary
print(df_update_teamdetails)



###Numerical Data Example for Handling Missing Values: MICE

Next, let's look at MICE: Multiple Imputation by Chained Equations.

This statistical technique is a form of multiple imputation, meaning that it creates multiple "complete" datasets by filling in the missing values multiple times. Then, these multiple datasets are analyzed separately, and the results are combined to produce more accurate and robust estimates (in comparison to single imputation methods).

This method has pros and cons:
Pros
1. Handles Missing at Random (MAR) Data: MICE is designed to work well when data is missing at random, meaning the probability of a value being missing depends on observed data but not on the missing value itself. This is a common scenario in many real-world datasets.
1. Preserves relationships: MICE takes into account the relationships between variables when imputing missing values, leading to more plausible and realistic imputations.
1. Accounts for Uncertainty: By creating multiple imputed datasets, MICE reflects the uncertainty inherent in the imputation process, leading to more accurate statistical inferences.

Cons:
1. The Assumptions with MICE: MICE assumes that the data is missing at random (MAR). If this assumption is violated, the imputations may be biased.
1. Compute costs:  MICE can be computationally intensive, especially for large datasets or when using complex predictive models.
1. Sensitive to Model Choice: The choice of predictive models within MICE can affect the quality of the imputations. Careful consideration is needed to select appropriate models for each variable

Overall, MICE is a good technique to know and and understand if the data is missing at random. But, it is not a jack-of-all trades and should only be used after carefully determining if it is the best technique to use. For instance, using MICE without understanding the computational costs (which is an outcome of data structure), would be problematic.


In [ ]:
"""
   This code is an example of the implementation of MICE on numerical data.
    """

"""
    How MICE Works:
     1.) Initialization: MICE starts by filling in all missing values with simple placeholders, such as the mean of the variable.
     2.) Iteration: The algorithm then cycles through each variable with missing data. For each variable:
          a.) It builds a predictive model (e.g., linear regression, logistic regression, or even more complex models) using the other variables in the dataset as predictors.
          b.) It uses this model to predict the missing values for that variable.
          c.) It replaces the placeholder values with these predicted values.
    3.) Repeat: Steps 2 is repeated multiple times (typically 5-10) until the imputed values converge or stabilize.
    4.) Multiple Datasets: The final result is multiple datasets, each with a slightly different set of imputed values, reflecting the uncertainty associated with the missing data.
    """
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
# Convert the dictionary to a DataFrame
df_update_teamdetailsMICE = pd.DataFrame(dataframes['team_details'])

# Initialize the MICE imputer
imputer = IterativeImputer()

# Impute missing values only in the 'arena_capacity' column
df_update_teamdetailsMICE['arenacapacity'] = imputer.fit_transform(df_update_teamdetailsMICE[['arenacapacity']])

# Display the updated DataFrame before converting back to a dictionary
print(df_update_teamdetailsMICE)



###One More Thing...
A fun exercise would be to compare the results of MICE and Mean for arenas that had missing values and see which one is closer to the real world (a good Google search will find the arena capacity)! Or are both answers so close (both to each other AND the actual answer) that it doesn't matter?

Hmmmm...

###Numerical Data Example: Outlier Detection and Treatment

Now, let's move on to detecting outliers and treating them in numerical data.

The belief that outliers need to be treated is up for debate: sometimes outliers point at important data points that should be kept in the data (for instance if sales were higher for 2 months, keeping this information would be useful to determine if this was a one-off occasion, with the possibility of attempting to implement the same scenario so that it contiues; to detect possible issues such as fraud).

In [ ]:
# @title Numerical Data Example for Outlier Detection and Treatment: Visualization and Statistical Methods

# Create DataFrames from dictionaries
df_team = pd.DataFrame(dataframes['team'])
df_game = pd.DataFrame(dataframes['game'])

# Merge DataFrames on 'id'
merged_df = pd.merge(df_team, df_game, left_on='id', right_on='team_id_home', how='right')

# Calculate z-scores for 'pts_away'
merged_df['z_score_pts_away'] = (merged_df['pts_away'] - merged_df['pts_away'].mean()) / merged_df['pts_away'].std()

# Identify outliers based on z-scores
z_score_threshold = 3
z_score_outliers = merged_df[(merged_df['z_score_pts_away'] > z_score_threshold) | (merged_df['z_score_pts_away'] < -z_score_threshold)]

# Print z-score outliers
print("Outliers based on z-score:")
print(z_score_outliers)

# Calculate quartiles and IQR for 'pts_away'
Q1 = merged_df['pts_away'].quantile(0.25)
Q3 = merged_df['pts_away'].quantile(0.75)
IQR = Q3 - Q1

# Identify outliers based on IQR
iqr_outliers = merged_df[(merged_df['pts_away'] < Q1 - 1.5 * IQR) | (merged_df['pts_away'] > Q3 + 1.5 * IQR)]

# Print IQR outliers
print("\nOutliers based on IQR:")
print(iqr_outliers)

# Create a boxplot of 'pts_away'
chart = alt.Chart(merged_df).mark_boxplot().encode(
    y='pts_away'
).properties(
    title='Boxplot of Away Points'
)

# Create a boxplot of 'pts_away' using Seaborn
sns.boxplot(y=merged_df['pts_away'])
plt.title('Boxplot of Away Points')
plt.show()


###Another Thing...

So this is fine and dandy, but how do you interpret this information? Well, here is how to understand the Z-Score, Interquartile Range (IQR), and Boxplot

Z-Score is a popular method that measures the distance between a data point and the mean in terms of standard deviations. A higher z-score indicates a greater difference from the mean and a greater difference from the mean indicates the data could be an outlier.
IQR:A method that calculates the difference between the first and third quartiles of the data, which represents the middle 50% (the mean).
Boxplot: A visual representation of the data that uses the IQR to identify outliers as data points outside the box (where the box is the data around the mean).

They all show what is outside of the mean, which would be something to note IF the data is assumed to have a linear distribution.

Now how do you interpret this? Well, try it yourself--what is your interpretation of the results?

In [ ]:
#@title Numerical Data Example for Outlier Detection and Treatment: Normalization

"""
   This code is an example of the implementation of normalization on numerical data.
    """
"""
  In particular, we are going to normalize the points data. This is because certain teams
    are, for better or worse, much better than others in certain areas (i.e, the Golden State Warriors Stephen
    Curry is the best shooter in basketball right now. No, I will not change this statement).
    This will skew results for modeling, so normalizing the data will help.
"""
# Create DataFrames from the dictionary
df_line_score = pd.DataFrame(dataframes['line_score'])

# Add a small constant to avoid log(0) errors
df_line_score['pts_away'] = df_line_score['pts_away'].apply(lambda x: x + 1 if x <= 0 else x)

# Apply log transformation
df_line_score['log_pts_away'] = np.log(df_line_score['pts_away'])

# Display the first 5 rows
print(df_line_score.head().to_markdown(index=False, numalign="left", stralign="left"))








### Numerical Data Example for Data Transformation and Standardization: Binning

Binning (also known as discretization or bucketing) is a useful technique when standardizing data. This technique is used to change continuous numerical data into discrete intervals or "bins.". What makes this such a useful technique is that it purposefully useful in doing both transformation (changing the form of the data) and standardization (adjusting the scale and distribution of the data) purposes (unlike the other examples, which may lean a little to either transformation or standardization, even if they can be used for both).

Pros:
1. Simplifies data: Binning reduces the complexity of continuous data, making it easier to understand and analyze.
1. Handles outliers: Reduces the influence of outliers by grouping them into extreme bins (note: an important question to ask is SHOULD outliers be handled this way).
1. Enhances visualization: Makes data easier to visualize through histograms or bar charts, revealing patterns that may not be apparent in the raw data.

Cons:
1. Information loss: Binning discards some information about the precise values within each bin.
1. Potential for bias: Inappropriate binning can introduce bias or obscure important patterns in the data.
1. Reduced predictive power: In some cases, binning can reduce the predictive power of models by discarding fine-grained information.

In [ ]:
"""
   This code is an example of the implementation of Binning on numerical data.
    """

"""
    Binning Strategy:
    1.) Define Bins: Determine the number of bins and their boundaries. This can be done based on domain knowledge, statistical methods (like equal width or equal frequency), or data visualization techniques.
    2.) Assign Values to Bins: Assign each data point to the corresponding bin based on its value.
    3.) Replace Values with Bin Labels: Replace the original numerical values with the corresponding bin labels or representative values (e.g., bin midpoints).
    """
# Create DataFrames from the dictionary
df_line_scorebins = pd.DataFrame(dataframes['line_score'])

# Define the desired number of bins or bin edges
num_bins = 4  # Adjust as needed

# Apply binning to the 'pts_away' column
df_line_scorebins['binned_pts_away'] = pd.cut(df_line_score['pts_away'], bins=num_bins, labels=False)

# Display the first 5 rows
print(df_line_scorebins.head().to_markdown(index=False, numalign="left", stralign="left"))



###And Another Thing...

What determines the number of bins to use? Well, there is not a right answer here. Some data professionals use a use a rule such that the relative error of the number of events in each bin is below a threshold. Others recommend reviewing analyses done within that particular field (in this notebook, that would be basketball) and use that binning.

In the above example, I used 4 bins--does this make sense? I recommend changing the number of bin values to see the result (and if it makes sense).

# II. Techniques by End Goal

The above section was quite detailed, and if you made this far, congrats! You may find everything after the first section a bit easier, since now we are going to look a techniques based on the end goal. What makes this a bit easier is that this is just another way to look at quite a few of the techniques mentioned above--it is just classifying them into new buckets (or "bins", if you will). This section will only contain one example section, since most of the techniques in this section has been covered in section I.

You can argue that data has 3 key end goals. Depending on the end goal, certain analyses will be done. And, based on those analyses, certain techniques will be used more often:

1. Model Development: If the end goal is model development, the the data cleansing techniques will generally focus on handlign missing values, outlier dection and treatment, data transformation and standardization, and feature engineering.
  *   Techniques: Imputation, scaling, normalization, encoding, feature engineering (selection and extraction)

2. Business Analytics: With this end goal, the focus is on data accuracy, consistency, and completeness (after all, you are trying to analyze a business, and most businesses follow certain rules and policies that keep things consistent until a major market shift).
  *   Techniques: Standardization, handling missing values (see the previous section for a list of techniques in the category)

3. Market Research: This end goal lends itself to techniques that focus on data quality and representativeness (think surveys, focus groups, etc.)
  *   Techniques: Outlier detection, weighting and sampling, data validation


Also, I am sure you noticed that categorical data was not covered in section I. That is because categories are a broader area and consists of features.

Since this section will cover feature engineering, with a focus on categorical features, it will only use categorical data (so categorical data cleansing techniques will be covered in this section).







### Model Development Example: Feature Engineering

For this example, we will focus on feature engineering, and in particular feature selection. Feature Engineering is the process of selecting, transforming, and creating new features (variables or attributes) from raw data to improve the performance of machine learning models. This is a crucial step in the pipeline of developing machine learning models because the quality of feature significantly impacts the models performance.

There are multiple feature engineering techniques, but below are a few:

1.Filter Methods:select features based on statistical properties, independent of any machine learning algorithm.These methods are computationally efficient; however, may overlook complex interactions between features. Examples of these methods include linear discrimiant analysis and analysis of variance (ANOVA).

2.Wrapper Methods: use machine learning algorithms as a black box to evaluate the performance of different feature subsets (this is done as a iterative process). These methods directly optimize feature selection for a chosen model, but there is a risk of overfitting if the search space is too large (and can be computationally expensive on large datasets).

3.Embedded Methods: incorporate feature selection as part of the model training process itself (usually through regularization techniques) and penalize the use of irrelevant features. One of the reasons to use these methods is that they automatically adjust feature importance based on the model's learning. However, they require quite a bit of stastical knowledge since they often require careful tuning of regularization parameters.

4.Dimensionality Reduction Techniques: transform the original features into a new, lower-dimensional space while preserving the most important information. The new features are used for modeling. These techniques do a great job in uncovering hidden patterns or relationships in data on one hand, but the new features may not be as interpretable as the original ones on the other. One of the most common techniques is principle component analysis (PCA).

In [ ]:
#@title Model Development Example using Filter Method: Principle Component Analysis
"""
   This code uses Principle Component Analysis (PCA) to feature engineer the data. PCA is a feature extraction method (combines and transforms
   the dataset's original features to produce new features that are linear
   combinations of the original ones). This is dimensionality reduction technique and is one of the more commone ones
    """

# Create DataFrames from the dictionary
df_game_PCA = pd.DataFrame(dataframes['game'])

# Select relevant columns
X = df_game_PCA.drop(['wl_home'], axis=1)  # Features (excluding 'wl')
y = df_game_PCA['wl_home']  # Target variable

# Impute missing values in the 'wl' column (assuming it's categorical)
imputer = SimpleImputer(strategy='most_frequent')  # Replace missing values with the most frequent category
y = imputer.fit_transform(y.values.reshape(-1, 1))

# Encode categorical variables (if any)
le = LabelEncoder()
for col in X.select_dtypes(include='object').columns:
    X[col] = le.fit_transform(X[col])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Impute missing values in the X_train and X_test
imputer = SimpleImputer(strategy='most_frequent') # Replace missing values with the most frequent
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Standardize features (important for PCA)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Fit PCA
pca = PCA() # You can specify n_components if you want to reduce to a specific number of dimensions
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Print explained variance ratio with feature names
print("Explained Variance Ratio:")
explained_variance_df = pd.DataFrame({'Feature': X.columns, 'Explained Variance Ratio': pca.explained_variance_ratio_})
print(explained_variance_df.to_markdown(index=False, numalign="left", stralign="left"))

# Print components with feature names

# Print components with feature names
print("\nPCA Components:")
components_df = pd.DataFrame(pca.components_, columns=X.columns)
print(components_df)


### AND Another Thing

Intrepretation of the results of a PCA analysis requires the examination of the magnitude and direction (positive or negative) of the coefficients for the orginal variables. The larger the absolute value of the coefficient, the more important the corresponding variable is in calculating the component. A good exercise is to look at the outputs above and try and intepret the results. Which variables are important and which ones are not?

# III. Techniques by Data Structure

This the final section of this notebook--well done for making it this far! In this section, the discussion now turns to an area that is a bit more traditional when discussing data cleansing methods: data structure.

Data structure is a specialized format for organizing, processing, retrieving and storing data. They are often classified based on the following characteristics:
1. Linear (sequential order) or Non-linear (unordered sequence)
1. Homogenous or heterogeneous (are all of the data items in a particular repository the same--homogenous--or different (heterogenous)
1. Static or dynamiic (this is about how the data structures are compiled). Static structures have fixed sizes, structures and memory locations at compile time while dyanmic structures have sizes, structures, and memory locations that can change depending on the use.


###But What about Data Types?

Data types are building blocks of data structures (think of data structures being the base, such as a concrete slab base, of a house and the data types the chemicals used to create the concrete). (Note: In quite a few areas, these data types are called primitive or base data types--both phrases are used interchangeably and mean the same thing). These types include:

* Boolean
* Interger
* Floating point numbers
* Fixed point numbers
* Character
* Pointers
* String

### Data Structure Classification+Data Type=Data Structures

Data structure classification and data types work in tandem to determine the data strucutre. Think of this as a hierarchy: start with the classification, then under each classification is data type, and the final level is the different data structures.

The type of data structure used is dependent on the type of operations that will be required/the kinds of algorithms that will be applied. Here is a list of the common nonprimitive data structures (by their definition, this also categorizes them into complex data structures):

Nonprimitive Data Structures
* Array
* Stack
* Queue
* Linked list
* Tree
* Heap
* Graph
* Trie
* Hash table

Starting at array and going to linked list, these are often classified as linear data structures. Everything after linked list is considered to be non-linear structures.

Primitive data structures have a much simplier hiearchy tree, since they only focus on the data type:
* Interger
* Float
* Character
* Boolean

###Okay, but How Does This Tie with Data Cleansing

Well, based on the concepts, we can also classify data cleansing techniques based on the type of data structure that they are commonly used (and may be most suitable) for:

Primitive Data Structures
1. Range constraints and validation ensures that values fall within a predefined valid range or satisfy specific criteria
  
  * Techniques: validate against pre-defined business rules and real-world examples; range checks

2. Type conversion and casting is converting data from one primitive type to another, ensuring compatibility and proper representation.
 * Techniques: label enconding; meta-data conversions (converting a string representation of something to a more useful data type, such as a float)

3. Outlier detection and handling identifies and addresses extreme or unusual values (outliers) that deviate significantly from the rest of the data (BUT SHOULD WE ADDRESS OUTLIERS?).
  * Techniques: statistical methods (i.e., z-score); spelling correction

Nonprimitive Data Structures
1. Structural validation and consistency checks ensures that the data adheres to a predefined structure or schema, maintaining consistency and preventing errors.
  * Techniques: referential integrity checks (for relational databases; this is when you check for that the foreign keys match primary keys); schema checks (for JSON or XML data; an example of this is the IsValid method with the JSON Schema).

2. Data normalization and standardization transforms data into a consistent format and scale, ensuring comparability and improving model performance.
  * Techniques: scaling, normalization.


3. Handling missing and inconsistent data addresses missing values and inconsistencies within complex data structures, ensuring data completeness and reliability.
(Note: well, well, well: look who's back)
  * Techniques: imputation techniques (mode, medium, range or MICE); deletion (if infrequency)

4. Text data cleaning and preprocessing (for text-based structures)
  * Techniques: lemming, stemming, removing stop words


### Data Structure Example: Scaling with Min-Max

For this example, we will use scaling. This technique involves transforming numerical features within a dataset to ensure they are on a similar or comparable scale.

This is often necessary because different features can have vastly different ranges or units of measurement, which can negatively impact the performance of machine learning algorithms.

Min-max scaling preseves the original data shape. There are other types of scaling techniques (such as standardization, Robust scaling, or normalization. Both standardization and normalization are mentioned in this notebook in secion I). In the below example, Min-Max is being used on a integer data structure




In [ ]:
# @title Data Structure Example using Scaling: Min-Max

# Create DataFrames from the dictionary
df_game_MinMax = pd.DataFrame(dataframes['game'])

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Apply Min-Max scaling to the 'pts_away' column
df_game_MinMax['scaled_pts_away'] = scaler.fit_transform(df_game_MinMax[['pts_away']])

# Display the first few rows of the DataFrame with the scaled column
print(df_game_MinMax[['pts_away', 'scaled_pts_away']].head().to_markdown(index=False, numalign="left", stralign="left"))


In [ ]:
#If the 'pts_away' column was an array, you would rewrite the above code as:

# Create DataFrames from the dictionary
df_game_MinMax_array = pd.DataFrame(dataframes['game'])

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Apply Min-Max scaling to the 'player_scores' column (assuming it contains arrays)
df_game_MinMax_array['scaled_player_scores'] = df_game['player_scores'].apply(
    lambda scores: scaler.fit_transform(np.array(scores).reshape(-1, 1)).flatten()
)

# Display the first few rows of the DataFrame with the original and scaled arrays
print(df_game_MinMax_array[['player_scores', 'scaled_player_scores']].head().to_markdown(index=False, numalign="left", stralign="left"))

# IV. Conclusion

In this notebook, we were able to dive deep into data cleansings. The notebook focused on different ways to categorize data cleansing techniques while also having some examples of implementations of those techniques.

Now, this is just the start. It is recommended that you go and study any methods that may not have been covered in detail here as well as to PRACTICE! Practicing these methods is one of the best ways to strengthen your understanding of these methods.